<H1>Coursera IBM Applied Datascience Capstone</H1>

<H2>1. Introduction</H2>

I'm going to examine the differences in trending venues between the two extremes of COVID-19 transmission. I'll compare Salt Lake City, Utah and Burlington, Vermont. Burlington is the largest city in Vermont with a population of just over 42,000. Currently, Burlington has one of the lowest transmission rate in the US. Chittenden county Vermont, where Burllington is located, has a infection rate of just over 15,000 per 1 million population. Salt Lake county, Utah has an infection rate of just under 90,000 per 1 million population.

Dispite the large difference in infection rates, the two cities have similar restrictions and mandates in place with regard to indoor and outdoor gatherings, face coverings, and sporting activities.

So, does a higher transmission rate impact which types of venues are trending in these two locations?

<H2>2. Data</H2>

Trending venue data will be obtained from FourSquare. Data on the infection rates of the two cities are obtained from the John's Hopkins' GitHub repository (https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv).

A majority of the analysis will center around the FourSquare data as the COVID data is only available at the county level. Being the largest cities in their respective counties their COVID numbers should match well those of the county as a whole.

I'll do a closer analysis of the top 10 trending venue categories in each city using inferential statistics to see how they differ, if at all. I'll also look at clustering of the venues to see if venue locations (e.g. city center vs outskirts) differ between cities.

In [ ]:
# The code was removed by Watson Studio for sharing.

<H2>3. Methods</H2>

<H2>4. Results</H2>

<H2>5. Discussion</H2>

<H2>6. Conclusion</H2>